# Week1

## Excercise 1.1

In [58]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from enum import Enum

class Turn(Enum):
    LEFT = 1
    RIGHT = 2
    ON_SEGMENT = 3
    ON_LINE = 4  # Colinear with the segment but not on the segment
    
class Point:
    x = 0
    y = 0
    
    def ToString(self):
        return str(self.x) + ", " + str(self.y)
    
class Line:
    p1 = Point()
    p2 = Point()
    
    def ToString(self):
        return "(" + self.p1.ToString() + "), (" + self.p2.ToString() + ")"
        
    def __repr__(self):
        return self.ToString()
    def __str__(self):
        return self.ToString()

In [63]:
import numpy as np

def between(e1, e2, test):
    return min(e1, e2) < test and test < max(e1, e2)

def FindTurn(line : Line, point : Point):
    m = np.array([[line.p1.x, line.p1.y, 1], [line.p2.x, line.p2.y, 1], [point.x, point.y, 1]])
    det = np.linalg.det(m)
    if det == 0:
        if between(line.p1.x, line.p2.x, point.x) and between(line.p1.y, line.p2.y, point.y):
            return Turn.ON_SEGMENT
        return Turn.ON_LINE
    elif det > 0:
        return Turn.LEFT
    return Turn.RIGHT

l = Line()
p = Point()
print(FindTurn(l, p))

Turn.ON_SEGMENT


In [64]:
import unittest

class TestNotebook(unittest.TestCase):

    def test_find_turn_example_1(self):
        l = Line()
        l.p1.x = -4
        l.p1.y = 4
        l.p2.x = 2
        l.p2.y = -2
        
        p = Point()
        
        p.x = 0
        p.y = 0
        self.assertEqual(FindTurn(l, p), Turn.ON_SEGMENT)
        
        p.x = 5
        p.y = -5
        self.assertEqual(FindTurn(l, p), Turn.ON_LINE)
        
        p.x = 1
        p.y = 1
        self.assertEqual(FindTurn(l, p), Turn.LEFT)
        
        p.x = -1
        p.y = -1
        self.assertEqual(FindTurn(l, p), Turn.RIGHT)


unittest.main(argv=[''], verbosity=2, exit=False)

test_find_turn_example_1 (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
